In [59]:
# Importamos pandas y numpy
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats

In [60]:
#Expecificamos ruta

url = r'https://raw.githubusercontent.com/cdeniaca/Group-7/76322e4482f3f9e2d42538a322aeb517e693a958/Poblacion_Activa.csv'

In [61]:
#Cargamos el df

población_activa_df = pd.read_csv(url, sep=';')


In [ ]:
#Imprimimos el df

print(población_activa_df)

In [ ]:
# Muestra los tipos de datos de cada columna
print(población_activa_df.dtypes)


In [64]:
# Reemplazamos el punto por una cadena vacía para eliminar el separador de miles
población_activa_df['Total'] = población_activa_df['Total'].str.replace('.', '', regex=False)

# Reemplazamos la coma por un punto para que pandas pueda interpretarlo como decimal
población_activa_df['Total'] = población_activa_df['Total'].str.replace(',', '.', regex=False)

In [ ]:
# Convertimos la columna a tipo numérico
población_activa_df['Total'] = pd.to_numeric(población_activa_df['Total'], errors='coerce')

# Verificamos el resultado
print(población_activa_df['Total'].head())


In [66]:
# Asegúrate de que la columna 'Total' exista en tu DataFrame
if 'Total' in población_activa_df.columns:
    población_activa_df['Total'] = población_activa_df['Total'] * 1000
else:
    print("La columna 'Total' no se encuentra en el DataFrame.")

In [67]:
# Crear una columna 'AÑO' con los primeros 4 caracteres de 'Periodo'
población_activa_df['AÑO'] = población_activa_df['Periodo'].str[:4]

# Crear una columna 'Trimestre' con los caracteres restantes de 'Periodo'
población_activa_df['Trimestre'] = población_activa_df['Periodo'].str[4:]

# Crear una columna 'Trimestre' con los caracteres restantes de 'Tip'



In [68]:
# Filtrar el DataFrame para que solo contenga los años de 2018 a 2023
años_filtrados = ['2018', '2019', '2020', '2021', '2022', '2023']
población_activa_df = población_activa_df[
    población_activa_df['AÑO'].isin(años_filtrados) &
    (población_activa_df['Comunidad autónoma'] != 'Total Nacional') &
    (población_activa_df['Relación con la actividad en el trimestre actual'] != 'Total (trimestre actual)') &
    (población_activa_df['Relación con la actividad en el trimestre anterior'] == 'Total (trimestre anterior)')
]

In [69]:
# Usa apply() con una función lambda para separar y quedarte solo con la segunda parte de la cadena
población_activa_df['Comunidad autónoma'] = población_activa_df['Comunidad autónoma'].apply(lambda x: x.split(' ', 1)[1] if ' ' in x else x)


In [ ]:
población_activa_df

In [ ]:
# Contar las filas duplicadas
duplicados = población_activa_df.duplicated().sum()
print(f"Número de filas duplicadas: {duplicados}")


In [ ]:
# Contar los valores nulos en cada columna
nulos = población_activa_df.isnull().sum()
print("Número de valores nulos en cada columna:")
print(nulos)


In [ ]:
# Filtrar las filas que tienen al menos un valor nulo
filas_nulas = población_activa_df[población_activa_df.isnull().any(axis=1)]

# Mostrar las filas con valores nulos
print("Filas con valores nulos:")
print(filas_nulas)



In [ ]:
print(población_activa_df['Comunidad autónoma'].unique())


In [ ]:
print(población_activa_df['Relación con la actividad en el trimestre actual'].unique())


In [ ]:
# Comprobar duplicados en las combinaciones de índice
duplicados = población_activa_df.duplicated(subset=['Periodo', 'Comunidad autónoma', 'AÑO', 'Trimestre', 'Relación con la actividad en el trimestre actual'], keep=False)
if duplicados.any():
    print("Hay duplicados en los datos:")
    print(población_activa_df[duplicados])
else:
    print("No hay duplicados, puedes usar pivot() sin problemas.")


In [77]:
# Eliminar duplicados
población_activa_df = población_activa_df.drop_duplicates(subset=['Periodo', 'Comunidad autónoma', 'AÑO', 'Trimestre', 'Relación con la actividad en el trimestre actual'])


In [ ]:
print(población_activa_df['Relación con la actividad en el trimestre actual'].unique())


In [79]:
# Crear un DataFrame pivotado
df_pivot = pd.pivot_table(
    población_activa_df,
    index=['Periodo', 'Comunidad autónoma', 'AÑO', 'Trimestre'],  # Índices de filas
    columns='Relación con la actividad en el trimestre actual',  # Nuevas columnas
    values='Total',  # Valores a pivotar
    aggfunc='sum',  # Función de agregación
    fill_value=0  # Opcional: reemplaza valores NaN por 0
)


In [80]:
# Restablecer el índice para convertirlo en columnas regulares
df_pivot_reset = df_pivot.reset_index()

In [ ]:
# Imprimir las columnas para verificar los nombres
print(df_pivot_reset.columns)

In [82]:
df_pivot = df_pivot_reset

In [ ]:
df_pivot

In [84]:
# Renombrar las columnas específicas en el DataFrame
df_pivot = df_pivot.rename(columns={
    'Inactivos (trimestre actual)': 'Inactivos',
    'Ocupados (trimestre actual)': 'Ocupados',
    'Parados (trimestre actual)': 'Parados'
})

In [85]:
# Crear la nueva columna "Total Actividad" 
df_pivot['Total Actividad'] = (
    df_pivot['Inactivos'].fillna(0) +  # Considerar nulos como 0
    df_pivot['Ocupados'].fillna(0) +   # Considerar nulos como 0
    df_pivot['Parados'].fillna(0)      # Considerar nulos como 0
)


In [86]:
df_pivot

Relación con la actividad en el trimestre actual,Periodo,Comunidad autónoma,AÑO,Trimestre,Inactivos,Ocupados,Parados,Total Actividad
0,2018T1,Andalucía,2018,T1,2965500.0,2967700.0,975700.0,6908900.0
1,2018T1,Aragón,2018,T1,457300.0,561100.0,73500.0,1091900.0
2,2018T1,"Asturias, Principado de",2018,T1,439500.0,387900.0,68600.0,896000.0
3,2018T1,"Balears, Illes",2018,T1,381800.0,488700.0,103100.0,973600.0
4,2018T1,Canarias,2018,T1,737400.0,879500.0,228400.0,1845300.0
...,...,...,...,...,...,...,...,...
451,2023T4,Melilla,2023,T4,29000.0,26100.0,11200.0,66300.0
452,2023T4,"Murcia, Región de",2023,T4,526800.0,678500.0,87800.0,1293100.0
453,2023T4,"Navarra, Comunidad Foral de",2023,T4,233500.0,302800.0,30900.0,567200.0
454,2023T4,País Vasco,2023,T4,815200.0,1024200.0,69800.0,1909200.0


In [87]:
#Dataframe Ocupados

In [88]:
print(df_pivot.columns)


Index(['Periodo', 'Comunidad autónoma', 'AÑO', 'Trimestre', 'Inactivos',
       'Ocupados', 'Parados', 'Total Actividad'],
      dtype='object', name='Relación con la actividad en el trimestre actual')


In [89]:
# Seleccionar las columnas deseadas de df_pivot
df_nuevo = df_pivot[['Comunidad autónoma', 'AÑO', 'Ocupados', 'Parados', 'Inactivos', 'Total Actividad']]



In [90]:
df_nuevo

Relación con la actividad en el trimestre actual,Comunidad autónoma,AÑO,Ocupados,Parados,Inactivos,Total Actividad
0,Andalucía,2018,2967700.0,975700.0,2965500.0,6908900.0
1,Aragón,2018,561100.0,73500.0,457300.0,1091900.0
2,"Asturias, Principado de",2018,387900.0,68600.0,439500.0,896000.0
3,"Balears, Illes",2018,488700.0,103100.0,381800.0,973600.0
4,Canarias,2018,879500.0,228400.0,737400.0,1845300.0
...,...,...,...,...,...,...
451,Melilla,2023,26100.0,11200.0,29000.0,66300.0
452,"Murcia, Región de",2023,678500.0,87800.0,526800.0,1293100.0
453,"Navarra, Comunidad Foral de",2023,302800.0,30900.0,233500.0,567200.0
454,País Vasco,2023,1024200.0,69800.0,815200.0,1909200.0


In [91]:
# Agrupar por "AÑO" y "Comunidad autónoma" y calcular la media de "Ocupados" y "Total Actividad"
df_nuevo_agrupado = df_nuevo.groupby(['AÑO', 'Comunidad autónoma']).mean().reset_index()



In [92]:
df_nuevo_agrupado

Relación con la actividad en el trimestre actual,AÑO,Comunidad autónoma,Ocupados,Parados,Inactivos,Total Actividad
0,2018,Andalucía,3030750.0,904625.0,2983100.0,6918475.0
1,2018,Aragón,573825.0,68250.0,452275.0,1094350.0
2,2018,"Asturias, Principado de",391900.0,61750.0,439825.0,893475.0
3,2018,"Balears, Illes",560175.0,72925.0,349600.0,982700.0
4,2018,Canarias,897200.0,225325.0,735800.0,1858325.0
...,...,...,...,...,...,...
109,2023,Melilla,26775.0,9725.0,29375.0,65875.0
110,2023,"Murcia, Región de",668625.0,97950.0,519850.0,1286425.0
111,2023,"Navarra, Comunidad Foral de",298725.0,33000.0,232575.0,564300.0
112,2023,País Vasco,995750.0,83375.0,818725.0,1897850.0


In [93]:
df_nuevo_agrupado.to_csv('Poblacion_Ocupada_Agrupado.csv', index=False)

In [94]:
df_pivot.to_csv('Poblacion_Activa_Limpia.csv', index=False)

In [95]:
#################################### NUEVO CSV #####################################################################

In [96]:
#Expecificamos ruta

url_teletrabajo = r'https://raw.githubusercontent.com/cdeniaca/Group-7/refs/heads/main/Poblacion_Teletrabajo.csv'

In [97]:
#Cargamos el df

población_teletrabajo_df = pd.read_csv(url_teletrabajo, sep=';')

In [98]:
# Reemplazamos el punto por una cadena vacía para eliminar el separador de miles
población_teletrabajo_df['Total'] = población_teletrabajo_df['Total'].str.replace('.', '', regex=False)

# Reemplazamos la coma por un punto para que pandas pueda interpretarlo como decimal
población_teletrabajo_df['Total'] = población_teletrabajo_df['Total'].str.replace(',', '.', regex=False)

In [99]:
# Convertimos la columna a tipo numérico
población_teletrabajo_df['Total'] = pd.to_numeric(población_teletrabajo_df['Total'], errors='coerce')


In [100]:
población_teletrabajo_df['Comunidades y Ciudades Autónomas'] = población_teletrabajo_df['Comunidades y Ciudades Autónomas'].apply(
    lambda x: x[1:] if x.startswith(' ') else x
)


In [101]:
población_teletrabajo_df['Teletrabajo'] = población_teletrabajo_df['Teletrabajo'].apply(
    lambda x: x[1:] if x.startswith(' ') else x
)

In [102]:
# Muestra los tipos de datos de cada columna
print(población_teletrabajo_df.dtypes)

Año                                   int64
Comunidades y Ciudades Autónomas     object
Teletrabajo                          object
Total                               float64
dtype: object


In [103]:
# Elimina filas donde 'Total' tenga NaN
población_teletrabajo_df = población_teletrabajo_df.dropna(subset=['Total'])


In [104]:
población_teletrabajo_df.rename(columns={'Comunidades y Ciudades Autónomas': 'Comunidad autónoma'}, inplace=True)


In [105]:
población_teletrabajo_df.rename(columns={'Año': 'AÑO'}, inplace=True)

In [106]:
print(población_teletrabajo_df['Comunidad autónoma'].unique())


['Andalucía' 'Aragón' 'Asturias, Principado de' 'Balears, Illes'
 'Canarias' 'Cantabria' 'Castilla y León' 'Castilla - La Mancha'
 'Cataluña' 'Comunitat Valenciana' 'Extremadura' 'Galicia'
 'Madrid, Comunidad de' 'Murcia, Región de' 'Navarra, Comunidad Foral de'
 'País Vasco' 'Rioja, La' 'Ceuta' 'Melilla' 'Castilla-La Mancha']


In [107]:
print(población_teletrabajo_df['Teletrabajo'].unique())

['Han teletrabajado'
 'No han teletrabajado aunque su trabajo les permitiría teletrabajar'
 'No han teletrabajado porque su trabajo no les permitiría teletrabajar'
 'No han teletrabajado']


In [108]:
# Crear un DataFrame pivotado
df_pivot_teletrabajo = pd.pivot_table(
    población_teletrabajo_df,
    index=['AÑO', 'Comunidad autónoma'],  # Asegúrate de usar el nombre correcto
    columns='Teletrabajo',
    values='Total',
    aggfunc='sum',
    fill_value=0
)


In [109]:
# Restablecer el índice para convertirlo en columnas regulares
df_pivot_teletrabajo = df_pivot_teletrabajo.reset_index()

In [110]:
df_pivot_teletrabajo

Teletrabajo,AÑO,Comunidad autónoma,Han teletrabajado,No han teletrabajado,No han teletrabajado aunque su trabajo les permitiría teletrabajar,No han teletrabajado porque su trabajo no les permitiría teletrabajar
0,2021,Andalucía,19.0,51.3,26.7,102.9
1,2021,Aragón,19.8,41.3,18.3,48.5
2,2021,"Asturias, Principado de",17.1,5.2,27.9,104.7
3,2021,"Balears, Illes",12.4,43.3,27.0,117.3
4,2021,Canarias,22.5,54.8,21.1,38.6
5,2021,Cantabria,10.1,47.5,4.4,111.0
6,2021,Castilla y León,6.9,45.7,22.2,80.2
7,2021,Castilla-La Mancha,12.3,42.8,24.4,120.6
8,2021,Cataluña,36.8,42.6,23.4,97.1
9,2021,Ceuta,9.2,66.3,65.6,58.8


In [111]:
# Convertir la columna 'AÑO' a tipo int en ambos DataFrames, o cambiar a str si prefieres.
df_nuevo_agrupado['AÑO'] = df_nuevo_agrupado['AÑO'].astype(int)
df_pivot_teletrabajo['AÑO'] = df_pivot_teletrabajo['AÑO'].astype(int)

# O convertir ambas a string
# df_nuevo_agrupado['AÑO'] = df_nuevo_agrupado['AÑO'].astype(str)
# población_teletrabajo_df['AÑO'] = población_teletrabajo_df['AÑO'].astype(str)


In [112]:
df_final_teletrabajo = pd.merge(df_nuevo_agrupado, df_pivot_teletrabajo, on=['Comunidad autónoma', 'AÑO'], how='inner')


In [113]:
df_final_teletrabajo


,AÑO,Comunidad autónoma,Ocupados,Parados,Inactivos,Total Actividad,Han teletrabajado,No han teletrabajado,No han teletrabajado aunque su trabajo les permitiría teletrabajar,No han teletrabajado porque su trabajo no les permitiría teletrabajar
0,2021,Andalucía,3152500.0,881400.0,3043700.0,7077600.0,19.0,51.3,26.7,102.9
1,2021,Aragón,588625.0,67800.0,467950.0,1124375.0,19.8,41.3,18.3,48.5
2,2021,"Asturias, Principado de",391475.0,55125.0,439275.0,885875.0,17.1,5.2,27.9,104.7
3,2021,"Balears, Illes",538000.0,89950.0,365450.0,993400.0,12.4,43.3,27.0,117.3
4,2021,Canarias,830425.0,254550.0,790500.0,1875475.0,22.5,54.8,21.1,38.6
5,2021,Cantabria,244975.0,31025.0,223875.0,499875.0,10.1,47.5,4.4,111.0
6,2021,Castilla y León,975975.0,128225.0,960100.0,2064300.0,6.9,45.7,22.2,80.2
7,2021,Cataluña,3490075.0,462075.0,2507400.0,6459550.0,36.8,42.6,23.4,97.1
8,2021,Ceuta,24350.0,11450.0,28600.0,64400.0,9.2,66.3,65.6,58.8
9,2021,Comunitat Valenciana,2076750.0,397875.0,1808600.0,4283225.0,23.2,5.4,27.1,99.4
